<a href="https://colab.research.google.com/github/Dawn-Of-Justice/Dawn-Of-Justice/blob/main/CodeYourGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 4.9 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
import numpy as np
import random
from transformers import GPT2Tokenizer,GPT2LMHeadModel,GPT2Model,GPT2Config
import torchvision
import requests
from PIL import Image
from io import BytesIO
from torch.utils.data import TensorDataset
import pandas as pd
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
import pandas as pd

filename = "SARA_DATA.txt"
df = pd.read_table(filename)

data = df['Data:'].values


In [ ]:
data[0:5]

In [ ]:
# df = pd.read_table("csi_chatbot_dat2.txt")
# bios = df['data:'].values

tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') #gpt2-medium

bios = data

input_id = []
attn_mask = []

# data = encodings_dict = tokenizer('<|startoftext|>'+ bios[0] + '<|endoftext|>', truncation=True, max_length=768, padding="max_length")

for i in range(len(bios)):
  #print(bios[i])
  encodings_dict = tokenizer('<|startoftext|>'+ bios[i] + '<|endoftext|>', truncation=True, max_length=768, padding="max_length")

  input_id.append(encodings_dict['input_ids'])
  attn_mask.append(encodings_dict['attention_mask'])
  # print(len(input_id))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print(len(input_id),len(attn_mask))
input_id = torch.tensor(input_id).long()
attn_mask = torch.tensor(attn_mask).long()

print(input_id.shape,attn_mask.shape)
print(input_id.dtype,attn_mask.dtype)


28 28
torch.Size([28, 768]) torch.Size([28, 768])
torch.int64 torch.int64


In [ ]:
# # Load the GPT tokenizer.
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') #gpt2-medium

# dataset = GPT2Dataset(df, tokenizer, max_length=768)

dataset = TensorDataset(input_id,attn_mask)

train_dataloader = DataLoader(
            dataset,  # The training samples. 
            batch_size = 2 # Trains with this batch size.
        )
data = enumerate(train_dataloader)
# encodings_dict = tokenizer('<|startoftext|>'+ sentences[1] + '<|endoftext|>',return_tensors='pt', truncation=True, max_length=10, padding="max_length")
# print(encodings_dict['input_ids'])
d,d1 = next(data)
print(d1[1].shape)

torch.Size([2, 768])


In [ ]:
import math
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/2)
print(total_samples,n_iterations)


28 14


In [ ]:
# I'm not really doing anything with the config buheret
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

# instantiate the model
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)

# this step is necessary because I've added some tokens (bos_token, etc) to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))


Embedding(50259, 768)

In [ ]:
epochs = 1009
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/3)
print(total_samples,n_iterations)

28 10


In [ ]:

epochs = 5
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8

# this produces sample output every 100 steps
sample_every = 100

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
total_steps = len(train_dataloader) * epochs


scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

In [ ]:
device = torch.device("cuda")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
epochs = 100

total_steps = len(train_dataloader) * epochs


scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

  

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        # print(b_labels)
        b_masks = batch[1].to(device)

        model.zero_grad()        

        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]  

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % 5 == 0 and not step == 0:

            # print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss))

            model.eval()

            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"Average train loss:{avg_train_loss}")
    if avg_train_loss < 0.02:
      print(f"Yay Got there at Epoch:{epoch_i} Step:{step} Average Loss:{avg_train_loss} Loss{loss.item()}")    
      break


======== Epoch 1 / 100 ========
Training...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0:  spend


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: plom
Average train loss:3.595524183341435

======== Epoch 2 / 100 ========
Training...


...

0:  DAHuman: Go to seminar1 on the seminar page AI: seminar/seminar.
Average train loss:0.021537945605814457

======== Epoch 10 / 100 ========
Training...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0:  interventions1 http


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0:  AubHuman: Take me to FAQ page AI: /faq.
Average train loss:0.014945413185549634
Yay Got there at Epoch:9 Step:13 Average Loss:0.014945413185549634 Loss0.012130149640142918


In [ ]:
model.eval()

prompt = f"""<|startoftext|> Human1: take me to about page"""
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

print(generated)

sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=300, 
                                max_length = 100,
                                top_p=0.95, 
                                num_return_sequences=5
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))